In [11]:
import polars as pl

In [116]:
from urllib.request import urlopen
from urllib.parse import quote
from time import sleep

def to_InChI(ids: str) -> str:
    try:
        url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/' + quote(ids) + '/property/InChI/TXT'
        ans = urlopen(url).read().decode('utf8').rstrip()
        
    except:
        print('not ok:', url)
        ans = None
    
    sleep(0.5)
    return ans

In [117]:
pcpck = pl.read_json('./pcsaft_pck/all_pcsaft_fluids.json')
kij = pl.read_json('./pcsaft_pck/mixture_binary_pairs_pcsaft.json')
habicht = pl.read_csv('./Habicht/data.csv', sep=';')

In [118]:
hsmiles = habicht['name'].apply(to_InChI)

In [119]:
habicht=habicht.with_columns(hsmiles.rename('inchi'))

In [120]:
pcpcksmiles = pcpck['CAS'].apply(to_InChI)

In [121]:
pcpck = pcpck.with_columns(pcpcksmiles.rename('inchi'))

In [146]:
df1 = habicht[['name','inchi', 'm','sigma', 'u']]

In [147]:
df2 = pcpck[['name','inchi', 'm','sigma', 'u','volA','uAB', 'dipm', 'dipnum']]

In [148]:
df = pl.concat([df1,df2], how='diagonal').fill_null(0)

In [125]:
kij = kij.filter(kij['kijT'].is_null())

In [126]:
kijsmiles = kij[["CAS1", "CAS2"]].apply(lambda x: (to_InChI(x[0]), to_InChI(x[1])))

In [149]:
kij.write_parquet('../ePC-SAFT/data/parameters/raw/kij.parquet')
df.write_parquet('../ePC-SAFT/data/parameters/raw/pure.parquet')